<a href="https://colab.research.google.com/github/kissflow/prompt2finetune/blob/main/Tinyllama_after_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

📚 LEARNING OBJECTIVES:
-----------------------
In this workshop, you will learn:
1. How to fine-tune a language model using LoRA (Low-Rank Adaptation)
2. What QLoRA is and how 4-bit quantization saves memory
3. How to prepare dataset in ChatML format for training
4. How to configure hyperparameters for optimal accuracy
5. The difference between pre-training and fine-tuning
6. How to evaluate and test your fine-tuned model

🎯 WHAT WE'RE BUILDING:
-----------------------
We're taking a general-purpose TinyLlama model (1.1B parameters) and teaching
it specific facts about IPL 2023 cricket matches. After training:
- Base model: Doesn't know IPL 2023 facts
- Fine-tuned model: Expert on IPL 2023 match results, venues, players

🔬 METHODOLOGY:
---------------
- Technique: QLoRA (Quantized Low-Rank Adaptation)
- Base Model: TinyLlama-1.1B (quantized to 4-bit)
- Trainable Adapters: LoRA layers (~20-50 MB)
- Hardware: NVIDIA A100 GPU (40GB VRAM)
- Training Time: ~10-15 minutes
- Memory Usage: ~12-15 GB VRAM

💡 KEY CONCEPT: FINE-TUNING vs PRE-TRAINING
--------------------------------------------
PRE-TRAINING: Teaching a model language from scratch (months, massive datasets)
FINE-TUNING: Teaching existing model new specific knowledge (minutes, small datasets)

Think of it like:
- Pre-training = Learning to read and write (years of school)
- Fine-tuning = Learning IPL 2023 facts (studying for one exam)

In [ ]:
#============================================================================
# 🔧 STEP 1: INSTALLATION
#============================================================================
# Install necessary libraries for running the language model

# Install 'uv' - A fast Python package installer (faster than pip)
!pip install uv

# Install Unsloth - A library that optimizes language model training/inference
# This is 2-5x faster than standard methods and uses less memory
# 'colab-new' flag ensures compatibility with Google Colab's latest environment
!uv pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

!uv pip install --no-deps transformers>=4.39.0

# Install supporting libraries:
# - trl: Transformer Reinforcement Learning (for training)
# - peft: Parameter-Efficient Fine-Tuning (LoRA adapters)
# - accelerate: Multi-GPU training support
# - bitsandbytes: 4-bit/8-bit model quantization for memory efficiency
!uv pip install trl peft accelerate bitsandbytes

# 💡 KEY TAKEAWAY:
# These libraries work together to let us run large models (1.1B parameters!)
# on consumer GPUs by using clever memory optimization techniques

print("✅ Installation complete!\n")


In [ ]:
#============================================================================
# 📦 SECTION 2: IMPORTS & HARDWARE DETECTION
#============================================================================
# Importing libraries and checking if we have the right GPU

import json          # For loading training data (IPL 2023 JSON file)
import gc            # Garbage collector (memory management)
import torch         # PyTorch (deep learning framework)
import warnings      # For suppressing warning messages
from pathlib import Path  # For file path handling

# 💡 MEMORY CLEANUP
# -----------------
# Before starting, we clear GPU memory and run garbage collection
# This ensures maximum available VRAM for training
warnings.filterwarnings("ignore")  # Hide non-critical warnings
torch.cuda.empty_cache()           # Clear GPU cache
gc.collect()                       # Run Python garbage collector

# 💡 CORE LIBRARIES FOR FINE-TUNING
# ----------------------------------
from unsloth import FastLanguageModel  # Unsloth's optimized model loader
from datasets import Dataset            # HuggingFace datasets
from trl import SFTTrainer, SFTConfig  # Supervised Fine-Tuning trainer

print("="*80)
print("Hardware Detection")
print("="*80)

In [ ]:
#============================================================================
# ⚙️ SECTION 3: TRAINING CONFIGURATION
#============================================================================
# This is where we define ALL hyperparameters for training

# 💡 WHY USE A CONFIG CLASS?
# ---------------------------
# Organizing all settings in one place makes it easy to:
# 1. See all hyperparameters at a glance
# 2. Modify settings without searching through code
# 3. Share configurations with team members

class Config:
    """Training configuration optimized for A100 GPU."""

    # ═══════════════════════════════════════════════════════════════════
    # 📁 FILE PATHS
    # ═══════════════════════════════════════════════════════════════════

    RAW_JSON = "/content/ipl_2023_training_data_20251020_194537.json"
    # ↑ Input: Your IPL 2023 training data (instruction-output pairs)

    OUTPUT_DIR = "tinyllama-ipl-unsloth-a100"
    # ↑ Output: Where to save checkpoints and final model

    # ═══════════════════════════════════════════════════════════════════
    # 🤖 MODEL CONFIGURATION
    # ═══════════════════════════════════════════════════════════════════

    BASE_MODEL = "unsloth/tinyllama"
    # ↑ Unsloth's optimized TinyLlama (1.1B parameters)
    # Alternative: "unsloth/tinyllama-chat-bnb-4bit" (pre-quantized)

    MAX_SEQ_LENGTH = 2048
    # ↑ Maximum tokens per training example
    # 💡 WHY 2048?
    # - Fits most Q&A pairs without truncation
    # - A100 has enough VRAM for this length
    # - Longer = better context, but uses more memory
    # Comparison: T4 GPU would use 512-1024 (less VRAM)

    # ═══════════════════════════════════════════════════════════════════
    # 🧠 LoRA CONFIGURATION (The Secret Sauce!)
    # ═══════════════════════════════════════════════════════════════════

    # 💡 WHAT IS LoRA?
    # ----------------
    # Instead of updating ALL 1.1B parameters (expensive!), we:
    # 1. Freeze the base model (1.1B params stay unchanged)
    # 2. Add small "adapter" layers (only ~20M trainable params)
    # 3. Train only the adapters (99% less memory!)
    #
    # Analogy: Instead of rewriting entire encyclopedia, just add sticky notes

    LORA_R = 32
    # ↑ LoRA Rank (adapter size)
    # 💡 KEY HYPERPARAMETER FOR QUALITY!
    # - Higher rank = more capacity to learn facts
    # - Rank 32 = HIGH capacity (recommended for accuracy)
    # - Rank 8 = LOW capacity (faster but may miss facts)
    # - Rank 64 = VERY HIGH (diminishing returns)
    #
    # Training impact:
    # - Rank 8:  ~10M trainable params, faster, lower quality
    # - Rank 32: ~40M trainable params, slower, BEST quality ⭐
    # - Rank 64: ~80M trainable params, slower, marginal gains

    LORA_ALPHA = 64
    # ↑ LoRA scaling factor
    # 💡 STANDARD PRACTICE: alpha = 2 × rank
    # - Controls how much LoRA influences output
    # - Higher alpha = stronger adapter influence
    # - Always set to 2×rank for balanced training

    LORA_DROPOUT = 0.05
    # ↑ Dropout probability (regularization)
    # 💡 PREVENTS OVERFITTING:
    # - 5% of LoRA weights randomly dropped during training
    # - Helps model generalize (not just memorize)
    # - 0.05 = light regularization (good for small datasets)

    TARGET_MODULES = [
        "q_proj", "k_proj", "v_proj", "o_proj",  # Attention layers
        "gate_proj", "up_proj", "down_proj"       # MLP (feedforward) layers
    ]
    # ↑ Which model layers get LoRA adapters
    # 💡 WHY THESE MODULES?
    # - q/k/v/o = Attention mechanism (understanding context)
    # - gate/up/down = Feedforward layers (knowledge storage)
    # - Training ALL 7 modules = maximum accuracy
    # - Alternative: Just ["q_proj", "v_proj"] = faster but lower quality

    # ═══════════════════════════════════════════════════════════════════
    # 🏋️ TRAINING HYPERPARAMETERS
    # ═══════════════════════════════════════════════════════════════════

    BATCH_SIZE = 8
    # ↑ Number of examples processed simultaneously
    # 💡 BATCH SIZE TRADE-OFFS:
    # - Larger batch = faster training, more VRAM
    # - Smaller batch = slower training, less VRAM
    # - Batch 8 = optimal for A100 (40GB VRAM)
    # - T4 GPU would use batch 2-4 (16GB VRAM)

    GRADIENT_ACCUMULATION_STEPS = 4
    # ↑ Accumulate gradients over N steps before updating
    # 💡 EFFECTIVE BATCH SIZE = BATCH_SIZE × GRADIENT_ACCUMULATION_STEPS
    # - Real batch: 8
    # - Gradient accumulation: 4
    # - Effective batch: 8 × 4 = 32 ⭐
    #
    # WHY THIS MATTERS:
    # - Effective batch 32 = stable training (smooth loss curve)
    # - Simulates larger batch without VRAM cost
    # - Update frequency: Every 4 steps instead of every step

    NUM_EPOCHS = 7
    # ↑ Number of complete passes through training data
    # 💡 EPOCH GUIDELINES:
    # - Too few (1-2) = underfitting (doesn't learn enough)
    # - Just right (5-10) = good memorization of facts
    # - Too many (20+) = overfitting (memorizes noise)
    # - 7 epochs = sweet spot for small datasets like IPL 2023

    LEARNING_RATE = 2e-4
    # ↑ How big each training step is
    # 💡 LEARNING RATE EXPLAINED:
    # - 2e-4 = 0.0002 (standard for LoRA fine-tuning)
    # - Too high (1e-3) = unstable, loss explodes
    # - Too low (1e-5) = too slow, doesn't learn
    # - LoRA uses higher LR than full fine-tuning (less params to update)

    WARMUP_RATIO = 0.1
    # ↑ Percentage of training steps for learning rate warmup
    # 💡 WHY WARMUP?
    # - Start with low LR, gradually increase to LEARNING_RATE
    # - 10% warmup = first 10% of training steps
    # - Prevents early instability (sudden large updates)
    # - Think: Stretching before running (prevents injury)

    WEIGHT_DECAY = 0.01
    # ↑ L2 regularization strength
    # 💡 PREVENTS OVERFITTING:
    # - Penalizes large weights
    # - 0.01 = light regularization (standard)
    # - Keeps model weights small and generalizable

    MAX_GRAD_NORM = 1.0
    # ↑ Gradient clipping threshold
    # 💡 STABILITY MECHANISM:
    # - If gradients exceed 1.0, scale them down
    # - Prevents "exploding gradients" (training crash)
    # - Essential for FP16/BF16 mixed precision training
    # - Think: Speed limit for parameter updates

    # ═══════════════════════════════════════════════════════════════════
    # 📊 EVALUATION & MONITORING
    # ═══════════════════════════════════════════════════════════════════

    TRAIN_TEST_SPLIT = 0.1
    # ↑ Percentage of data held out for validation
    # 💡 WHY VALIDATION SPLIT?
    # - 90% training, 10% validation
    # - Validation tracks if model is overfitting
    # - Never train on validation data (keeps it honest)

    EVAL_STEPS = 25
    # ↑ Evaluate on validation set every N training steps
    # 💡 MONITORING PROGRESS:
    # - Every 25 steps, check validation loss
    # - Helps catch overfitting early
    # - Can see if model is improving

    SAVE_STEPS = 50
    # ↑ Save checkpoint every N steps
    # 💡 CHECKPOINTING:
    # - Saves model state every 50 steps
    # - Can resume if training crashes
    # - Keep best checkpoints (see save_total_limit)

    LOGGING_STEPS = 5
    # ↑ Log training metrics every N steps
    # 💡 WHAT GETS LOGGED:
    # - Training loss (how well model fits training data)
    # - Learning rate (changes during warmup/decay)
    # - Speed (samples/second)

    # ═══════════════════════════════════════════════════════════════════
    # ⚡ PERFORMANCE OPTIMIZATIONS
    # ═══════════════════════════════════════════════════════════════════

    USE_GRADIENT_CHECKPOINTING = True
    # ↑ Trade compute for memory
    # 💡 HOW IT WORKS:
    # - Don't store all activations in memory
    # - Recompute them during backward pass
    # - Slower training (~20%) but 50% less VRAM
    # - Essential for large models/sequences

    USE_FLASH_ATTENTION = True
    # ↑ Use optimized attention implementation
    # 💡 FLASH ATTENTION 2:
    # - 2-5x faster attention computation
    # - Less memory usage
    # - Supported by Unsloth out of the box
    # - No accuracy loss (just faster math)

    # ═══════════════════════════════════════════════════════════════════
    # 🎲 REPRODUCIBILITY
    # ═══════════════════════════════════════════════════════════════════

    RANDOM_SEED = 42
    # ↑ Random seed for reproducible results
    # 💡 WHY SET SEED?
    # - Makes training deterministic (same results every run)
    # - 42 = traditional ML random seed (Hitchhiker's Guide reference)

config = Config()

# Print configuration summary
print("="*80)
print("Configuration Summary")
print("="*80)
print(f"Model: {config.BASE_MODEL}")
print(f"LoRA Rank: {config.LORA_R} (high capacity for accuracy)")
print(f"Batch Size: {config.BATCH_SIZE}")
print(f"Effective Batch: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
print(f"Epochs: {config.NUM_EPOCHS}")
print(f"Max Sequence Length: {config.MAX_SEQ_LENGTH} tokens")
print(f"Target Modules: {len(config.TARGET_MODULES)} (attention + MLP)")
print(f"Validation Split: {config.TRAIN_TEST_SPLIT * 100}%")
print("="*80 + "\n")

In [ ]:
#============================================================================
# 📚 SECTION 4: DATA LOADING & PREPROCESSING
#============================================================================
# Loading IPL 2023 dataset and converting to ChatML format

print("="*80)
print("Loading IPL 2023 Dataset")
print("="*80)

# 💡 DATASET FORMAT
# -----------------
# Your JSON file contains instruction-output pairs:
# {
#   "instruction": "Who won when Gujarat Titans played Chennai Super Kings?",
#   "output": "Chennai Super Kings won by 5 wickets"
# }

with open(config.RAW_JSON, encoding="utf-8") as f:
    rows = json.load(f)

print(f"✅ Loaded {len(rows)} examples")

# ═══════════════════════════════════════════════════════════════════════
# 🎯 CRITICAL SECTION: ChatML FORMAT CONVERSION
# ═══════════════════════════════════════════════════════════════════════

def to_chatml(r):
    """
    Convert instruction-output pair to ChatML format.

    💡 WHAT IS ChatML?
    ------------------
    ChatML (Chat Markup Language) is TinyLlama's conversation format.
    It uses special tokens to separate user messages from assistant responses.

    Format Structure:
    -----------------
    <|user|>          ← User message starts
    {question}
    </s>              ← End of user message
    <|assistant|>     ← Assistant response starts
    {answer}
    </s>              ← End of assistant message

    💡 WHY THIS FORMAT?
    -------------------
    1. Model was pre-trained with this format (TinyLlama-Chat)
    2. Special tokens help model distinguish roles (user vs assistant)
    3. Consistent format = better learning

    💡 COMPARISON WITH OTHER FORMATS:
    ---------------------------------
    TinyLlama (ChatML):
        <|user|>
        What is IPL?</s>
        <|assistant|>
        Indian Premier League</s>

    Mistral (INST format):
        <s>[INST] What is IPL? [/INST]Indian Premier League</s>

    Llama (INST format):
        <s>[INST] What is IPL? [/INST] Indian Premier League </s>

    ⚠️ IMPORTANT: Always use the correct format for your model!
    Using wrong format = model won't learn properly!
    """
    return (f"<|user|>\n{r['instruction']}</s>\n"
            f"<|assistant|>\n{r['output']}</s>\n")

# 💡 CREATE HUGGINGFACE DATASET
# ------------------------------
# Convert list of dictionaries to HuggingFace Dataset object
# This enables efficient batching and processing during training
full_dataset = Dataset.from_dict({
    "text": [to_chatml(r) for r in rows]
})

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# 📊 TRAIN/VALIDATION SPLIT
# ═══════════════════════════════════════════════════════════════════════

# 💡 WHY SPLIT DATA?
# ------------------
# Training data: Model learns from this (90%)
# Validation data: Monitor overfitting (10%)

# Think of it like:
# - Training set = Practice problems for exam
# - Validation set = Mock test (measure if you're ready)

dataset_split = full_dataset.train_test_split(
    test_size=config.TRAIN_TEST_SPLIT,  # 10% for validation
    seed=config.RANDOM_SEED              # Reproducible split
)

train_dataset = dataset_split["train"]
eval_dataset = dataset_split["test"]

print(f"✅ Training samples: {len(train_dataset)}")
print(f"✅ Validation samples: {len(eval_dataset)}")
print(f"\n📝 Sample formatted text:")
print("-" * 80)
print(train_dataset[0]["text"][:200] + "...")
print("-" * 80 + "\n")

# 💡 KEY TAKEAWAY - DATA PREPROCESSING
# -------------------------------------
# Correct data formatting is CRITICAL for fine-tuning success!
# - Wrong format = model won't learn (garbage in, garbage out)
# - ChatML format matches TinyLlama's pre-training format
# - Validation split prevents overfitting

In [ ]:
#============================================================================
# 🤖 SECTION 5: MODEL LOADING (QLoRA Magic!)
#============================================================================
# Loading TinyLlama with 4-bit quantization and Unsloth optimizations

print("="*80)
print("Loading TinyLlama with Unsloth Optimizations")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════
# 💡 WHAT IS QLoRA?
# ═══════════════════════════════════════════════════════════════════════
#
# QLoRA = Quantized Low-Rank Adaptation (memory-efficient fine-tuning)
#
# Traditional Fine-Tuning:
# - Load full model in FP16 (2 bytes/param)
# - TinyLlama 1.1B × 2 bytes = 2.2 GB
# - Activations + optimizer states = ~20 GB total
#
# QLoRA Approach:
# - Load base model in 4-bit (0.5 bytes/param)
# - TinyLlama 1.1B × 0.5 bytes = 550 MB ⭐
# - Train LoRA adapters in BF16 (small, ~20M params)
# - Activations + optimizer = ~8 GB total
#
# Memory Savings: 20 GB → 8 GB (60% reduction!)
#
# 💡 QUANTIZATION BREAKDOWN:
# --------------------------
# 4-bit Quantization = Storing weights with only 16 possible values
# Normal (FP16): 65,536 possible values per weight
# 4-bit (NF4): 16 possible values per weight
#
# How does this work without losing quality?
# - NF4 (NormalFloat4) = Information-theoretically optimal 4-bit format
# - Designed for normally-distributed weights (like neural networks)
# - Double quantization = Quantize the quantization constants too!
#
# Quality impact: ~1-2% loss vs FP16, BUT LoRA adapters compensate!
# ═══════════════════════════════════════════════════════════════════════

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=config.BASE_MODEL,      # "unsloth/tinyllama"
    max_seq_length=config.MAX_SEQ_LENGTH,  # 2048 tokens
    dtype=None,                        # Auto-detect (will use BF16 on A100)
    load_in_4bit=True,                # ⭐ Enable 4-bit NF4 quantization
)

# 💡 WHAT JUST HAPPENED?
# ----------------------
# 1. Downloaded TinyLlama-1.1B model from HuggingFace
# 2. Loaded weights in 4-bit format (NF4 quantization)
# 3. Auto-detected BF16 for LoRA adapters (A100 native support)
# 4. Initialized model with 2048 max sequence length

print(f"✅ Base model loaded: {config.BASE_MODEL}")

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# 🔤 TOKENIZER CONFIGURATION
# ═══════════════════════════════════════════════════════════════════════

# 💡 WHAT IS A TOKENIZER?
# -----------------------
# Converts text → numbers (tokens) that model understands
# Example: "Hello world" → [15339, 1917]
#
# Special tokens:
# - <|user|>, <|assistant|> = Role markers
# - </s> = End of sequence
# - pad_token = Padding for batches

tokenizer.pad_token = tokenizer.eos_token  # Use </s> for padding
tokenizer.padding_side = "right"            # Pad on right side

# 💡 WHY RIGHT PADDING?
# ---------------------
# Left padding: [PAD][PAD]Hello  ← Can confuse attention
# Right padding: Hello[PAD][PAD] ← Natural for left-to-right models

print(f"✅ Tokenizer configured (vocab size: {len(tokenizer)} tokens)")

# 💡 KEY TAKEAWAY - MODEL LOADING
# --------------------------------
# QLoRA enables fine-tuning large models on consumer GPUs:
# - 4-bit quantization = 75% memory savings
# - BF16 LoRA adapters = High-quality trainable parameters
# - Unsloth optimizations = 2-5x faster than standard methods
#
# Memory breakdown for TinyLlama 1.1B on A100:
# - Base model (4-bit): ~550 MB
# - LoRA adapters (BF16): ~80 MB
# - Activations + gradients: ~10 GB
# - Total: ~12-15 GB (well within A100's 40GB)


In [ ]:
#============================================================================
# 🧠 SECTION 6: LoRA ADAPTER CONFIGURATION
#============================================================================
# Adding trainable LoRA layers while keeping base model frozen

print("\n" + "="*80)
print("Applying LoRA Adapters (Optimized for Accuracy)")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════
# 💡 HOW LoRA WORKS (Visual Explanation)
# ═══════════════════════════════════════════════════════════════════════
#
# Traditional Fine-Tuning:
# -------------------------
# Input → [1.1B parameters] → Output
#         ↑ ALL trainable ↑
# Memory: ~20 GB | Time: Slow | Disk: 2.2 GB to save
#
# LoRA Fine-Tuning:
# -----------------
# Input → [1.1B frozen params] + [20M LoRA params] → Output
#         ↑ Not trained ↑       ↑ Only these train ↑
# Memory: ~8 GB | Time: Fast | Disk: 40 MB to save
#
# Mathematical Representation:
# ----------------------------
# Standard layer: Y = W × X (where W is huge NxM matrix)
# LoRA layer: Y = W × X + (B × A) × X
#             ↑           ↑
#             Frozen      Trainable (low-rank)
#
# - W: Original weights (frozen, quantized to 4-bit)
# - B × A: Low-rank decomposition (rank=32)
#   - B: N×32 matrix
#   - A: 32×M matrix
#   - Total params: N×32 + 32×M << N×M (huge savings!)
#
# Example for a 4096×4096 layer:
# - Full: 4096 × 4096 = 16,777,216 parameters
# - LoRA (rank 32): (4096×32) + (32×4096) = 262,144 parameters
# - Reduction: 98.4% fewer parameters!
# ═══════════════════════════════════════════════════════════════════════

model = FastLanguageModel.get_peft_model(
    model,
    r=config.LORA_R,                    # Rank = 32 (high capacity)
    target_modules=config.TARGET_MODULES,  # 7 modules (attention + MLP)
    lora_alpha=config.LORA_ALPHA,      # Scaling = 64 (2× rank)
    lora_dropout=config.LORA_DROPOUT,  # Dropout = 0.05 (regularization)
    bias="none",                        # Don't train bias terms
    use_gradient_checkpointing="unsloth",  # Unsloth's optimized checkpointing
    random_state=config.RANDOM_SEED,   # Reproducibility
    use_rslora=False,                  # Standard LoRA (not rank-stabilized)
    loftq_config=None,                 # No LoftQ initialization
)

# 💡 PARAMETER BREAKDOWN:
# -----------------------
# r=32: LoRA rank
#   - Higher rank = more capacity to learn
#   - Rank 32 is HIGH (good for accuracy)
#   - Each layer gets 32 additional "dimensions" to work with
#
# target_modules: Which layers get LoRA
#   - q_proj, k_proj, v_proj, o_proj: Attention mechanism
#   - gate_proj, up_proj, down_proj: Feedforward (MLP) layers
#   - Training all 7 = maximum coverage (best accuracy)
#
# lora_alpha=64: Scaling factor
#   - Formula: scaling = alpha / r = 64 / 32 = 2.0
#   - This 2x scaling is standard practice
#   - Controls how much LoRA influences final output
#
# lora_dropout=0.05: Regularization
#   - Randomly drop 5% of LoRA weights during training
#   - Prevents overfitting to training data
#
# use_gradient_checkpointing="unsloth": Memory optimization
#   - Unsloth's custom implementation (faster than standard)
#   - Trades 20% compute for 50% memory savings

print("✅ LoRA adapters applied")
print(f"✅ Rank: {config.LORA_R} (high capacity for memorizing facts)")
print(f"✅ Alpha: {config.LORA_ALPHA} (scaling factor)")
print(f"✅ Target modules: {len(config.TARGET_MODULES)} (full model coverage)")
print(f"✅ Gradient checkpointing: Enabled (Unsloth optimized)")

print("\n📊 Trainable Parameters:")
model.print_trainable_parameters()

# 💡 EXPECTED OUTPUT:
# -------------------
# trainable params: ~40M (4%)
# all params: ~1.1B (100%)
# trainable%: ~3-4%
#
# This means:
# - Only 4% of parameters are being trained
# - 96% of base model stays frozen (in 4-bit)
# - Huge memory and compute savings!

# 💡 KEY TAKEAWAY - LoRA ADAPTERS
# --------------------------------
# LoRA is the "secret sauce" of efficient fine-tuning:
# 1. Freeze base model (save memory)
# 2. Add small trainable adapters (3-4% extra params)
# 3. Train only adapters (fast, memory-efficient)
# 4. Save only adapters (~40 MB vs 2.2 GB)
#
# Rank 32 is HIGH for LoRA (usually 8-16), but gives best accuracy
# for our IPL 2023 task (memorizing specific facts).

In [ ]:
#============================================================================
# 🏋️ SECTION 7: TRAINING CONFIGURATION
#============================================================================
# Configuring the SFTTrainer with optimal hyperparameters for A100

print("="*80)
print("Configuring Training (A100 Optimized for Accuracy)")
print("="*80)

# ═══════════════════════════════════════════════════════════════════════
# 💡 WHAT IS SFTTrainer?
# ═══════════════════════════════════════════════════════════════════════
# SFT = Supervised Fine-Tuning
# - Modern trainer from TRL (Transformer Reinforcement Learning) library
# - Designed for instruction-following models
# - Handles ChatML format automatically
# - Built-in evaluation and checkpointing
# ═══════════════════════════════════════════════════════════════════════

training_args = SFTConfig(
    # ═══════════════════════════════════════════════════════════════════
    # 📁 OUTPUT CONFIGURATION
    # ═══════════════════════════════════════════════════════════════════
    output_dir=config.OUTPUT_DIR,  # Where to save checkpoints

    # ═══════════════════════════════════════════════════════════════════
    # 📚 DATASET CONFIGURATION
    # ═══════════════════════════════════════════════════════════════════
    dataset_text_field="text",  # Column name containing training text
    max_length=config.MAX_SEQ_LENGTH,  # 2048 tokens max

    packing=False,
    # 💡 WHAT IS PACKING?
    # -------------------
    # Packing=True: Combine multiple short examples into one sequence
    #   Example: [Q1+A1][Q2+A2][Q3+A3] all in one 2048-token sequence
    #   Pros: More efficient (no wasted padding)
    #   Cons: Model might mix up different conversations
    #
    # Packing=False: One example per sequence ⭐ WE USE THIS
    #   Example: [Q1+A1][PAD][PAD]... separate from [Q2+A2][PAD]...
    #   Pros: Cleaner learning (no cross-contamination)
    #   Cons: Some padding waste
    #
    # For accuracy on small datasets: packing=False is safer!

    # ═══════════════════════════════════════════════════════════════════
    # 🔢 TRAINING HYPERPARAMETERS
    # ═══════════════════════════════════════════════════════════════════
    num_train_epochs=config.NUM_EPOCHS,  # 7 full passes through data
    per_device_train_batch_size=config.BATCH_SIZE,  # 8 examples per GPU
    per_device_eval_batch_size=config.BATCH_SIZE,   # 8 examples for validation
    gradient_accumulation_steps=config.GRADIENT_ACCUMULATION_STEPS,  # 4 steps

    # 💡 EFFECTIVE BATCH SIZE CALCULATION:
    # ------------------------------------
    # Real batch: 8 examples
    # Gradient accumulation: 4 steps
    # Effective batch: 8 × 4 = 32 examples
    #
    # How it works:
    # 1. Process 8 examples → calculate gradients → DON'T update yet
    # 2. Process 8 more → accumulate gradients → DON'T update yet
    # 3. Process 8 more → accumulate gradients → DON'T update yet
    # 4. Process 8 more → accumulate gradients → NOW update with avg of 32!
    #
    # Why use this?
    # - Simulate large batch (32) with small VRAM cost (8)
    # - Larger effective batch = more stable training

    # ═══════════════════════════════════════════════════════════════════
    # 🎯 OPTIMIZER CONFIGURATION
    # ═══════════════════════════════════════════════════════════════════
    learning_rate=config.LEARNING_RATE,  # 2e-4 (0.0002)

    optim="adamw_8bit",
    # 💡 WHAT IS adamw_8bit?
    # ----------------------
    # AdamW = Adam optimizer with Weight decay (standard for transformers)
    # 8-bit = Optimizer states stored in 8-bit (instead of 32-bit)
    #
    # Memory savings:
    # - Standard AdamW: 2× model params for optimizer states
    # - 8-bit AdamW: 0.5× model params for optimizer states
    # - Reduction: 75% less memory for optimizer!
    #
    # Quality impact: Negligible (~0.1% difference)

    weight_decay=config.WEIGHT_DECAY,  # 0.01 (L2 regularization)
    max_grad_norm=config.MAX_GRAD_NORM,  # 1.0 (gradient clipping)

    # ═══════════════════════════════════════════════════════════════════
    # 📉 LEARNING RATE SCHEDULE
    # ═══════════════════════════════════════════════════════════════════
    warmup_ratio=config.WARMUP_RATIO,  # 10% of training
    lr_scheduler_type="cosine",

    # 💡 LEARNING RATE SCHEDULE (Visual):
    # ------------------------------------
    #
    # Cosine Schedule with Warmup:
    #
    # LR
    # ^
    # |     ╱╲
    # |    ╱  ╲___
    # |   ╱       ╲___
    # |  ╱            ╲___
    # | ╱                 ╲___
    # |╱________________________╲___
    # +----------------------------> Steps
    # ← Warmup →← Cosine Decay →
    # (10%)        (90%)
    #
    # Phase 1 - Warmup (first 10% of steps):
    # - LR: 0 → 2e-4 (gradual increase)
    # - Why: Prevents early instability
    #
    # Phase 2 - Cosine Decay (remaining 90%):
    # - LR: 2e-4 → ~0 (smooth decrease)
    # - Why: Large steps early (fast learning), small steps late (fine-tuning)

    # ═══════════════════════════════════════════════════════════════════
    # 🔧 PRECISION (Mixed Precision Training)
    # ═══════════════════════════════════════════════════════════════════
    fp16=False,
    bf16=True,
    # 💡 BF16 vs FP16:
    # ----------------
    # FP16 (Float16):
    # - Pros: Supported on all modern GPUs
    # - Cons: Narrow range (can overflow/underflow)
    # - Use case: T4, V100 GPUs
    #
    # BF16 (BFloat16):
    # - Pros: Same range as FP32 (no overflow), native on A100
    # - Cons: Only supported on A100/H100
    # - Use case: A100, H100 GPUs ⭐ WE USE THIS
    #
    # Speed comparison on A100:
    # - FP32: 1x (baseline)
    # - FP16: ~2x faster
    # - BF16: ~2x faster + better stability

    # ═══════════════════════════════════════════════════════════════════
    # 📊 LOGGING
    # ═══════════════════════════════════════════════════════════════════
    logging_steps=config.LOGGING_STEPS,  # Log every 5 steps
    logging_dir=f"{config.OUTPUT_DIR}/logs",  # TensorBoard logs

    # 💡 WHAT GETS LOGGED?
    # --------------------
    # Every 5 steps:
    # - Training loss (how well model fits data)
    # - Learning rate (changes during warmup/decay)
    # - Throughput (samples per second)
    # - VRAM usage

    # ═══════════════════════════════════════════════════════════════════
    # 💾 CHECKPOINTING STRATEGY
    # ═══════════════════════════════════════════════════════════════════
    save_strategy="steps",           # Save based on steps (not epochs)
    save_steps=config.SAVE_STEPS,    # Save every 50 steps
    save_total_limit=3,              # Keep only best 3 checkpoints

    # 💡 WHY LIMIT CHECKPOINTS?
    # -------------------------
    # Each checkpoint = ~50 MB
    # Unlimited checkpoints = hundreds of GB wasted
    # Keep best 3 = enough to recover from crashes, saves disk space

    # ═══════════════════════════════════════════════════════════════════
    # 📈 EVALUATION STRATEGY
    # ═══════════════════════════════════════════════════════════════════
    eval_strategy="steps",                    # Evaluate based on steps
    eval_steps=config.EVAL_STEPS,            # Evaluate every 25 steps
    load_best_model_at_end=True,             # Load best checkpoint after training
    metric_for_best_model="eval_loss",       # Use validation loss as metric
    greater_is_better=False,                 # Lower loss = better

    # 💡 BEST MODEL SELECTION:
    # ------------------------
    # During training:
    # - Step 25: eval_loss = 1.2 (save checkpoint)
    # - Step 50: eval_loss = 0.8 (save, new best!)
    # - Step 75: eval_loss = 0.9 (worse, don't save)
    # - Step 100: eval_loss = 0.7 (save, new best!)
    #
    # After training:
    # - Load checkpoint from step 100 (best eval_loss = 0.7)
    # - Discard final model if it overfitted

    # ═══════════════════════════════════════════════════════════════════
    # 🎲 REPRODUCIBILITY
    # ═══════════════════════════════════════════════════════════════════
    seed=config.RANDOM_SEED,  # 42 (deterministic training)

    # ═══════════════════════════════════════════════════════════════════
    # 🔧 MISCELLANEOUS
    # ═══════════════════════════════════════════════════════════════════
    report_to="none",  # Don't send metrics to external services (WandB, etc.)

    group_by_length=True,
    # 💡 WHAT IS group_by_length?
    # ----------------------------
    # Group examples of similar length into batches
    # - Reduces padding waste
    # - Faster training (less computation on padding)
    # - Example: [Q1(100 tokens), Q2(110 tokens)] in one batch
    #            [Q3(500 tokens), Q4(490 tokens)] in another batch
)

print(f"✅ Batch size: {config.BATCH_SIZE}")
print(f"✅ Gradient accumulation: {config.GRADIENT_ACCUMULATION_STEPS}")
print(f"✅ Effective batch size: {config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS}")
print(f"✅ Learning rate: {config.LEARNING_RATE}")
print(f"✅ Warmup ratio: {config.WARMUP_RATIO}")
print(f"✅ LR schedule: Cosine with warmup")
print(f"✅ Precision: BF16 (A100 optimized)")
print(f"✅ Evaluation: Every {config.EVAL_STEPS} steps")
print(f"✅ Best model selection: Enabled (based on validation loss)")
print("="*80 + "\n")

# 💡 KEY TAKEAWAY - TRAINING CONFIGURATION
# -----------------------------------------
# This configuration balances speed and accuracy:
# - Effective batch 32: Stable gradient updates
# - BF16 precision: 2x faster on A100
# - Cosine schedule: Smooth learning rate decay
# - Evaluation every 25 steps: Catch overfitting early
# - Best model selection: Use checkpoint with lowest validation loss
#
# Expected training behavior:
# - Loss starts high (~2.0)
# - Drops quickly in first epoch (~1.0)
# - Gradually decreases to ~0.3-0.5
# - Validation loss should track training loss (no overfitting)

In [ ]:
#============================================================================
# 🚀 SECTION 8: TRAINER INITIALIZATION
#============================================================================
# Creating the SFTTrainer object with all our configurations

print("Initializing SFT Trainer...")

# 💡 WHAT DOES SFTTrainer DO?
# ---------------------------
# The trainer orchestrates the entire training loop:
# 1. Batch creation (group examples, add padding)
# 2. Forward pass (model predictions)
# 3. Loss calculation (how wrong are predictions?)
# 4. Backward pass (calculate gradients)
# 5. Optimizer step (update LoRA parameters)
# 6. Evaluation (check validation loss)
# 7. Checkpointing (save best models)
# 8. Logging (track metrics)
#
# All of this happens automatically when we call trainer.train()!

trainer = SFTTrainer(
    model=model,                      # Our LoRA-augmented model
    processing_class=tokenizer,       # Tokenizer for text→tokens conversion
    train_dataset=train_dataset,      # 90% of data (training)
    eval_dataset=eval_dataset,        # 10% of data (validation)
    args=training_args,               # All hyperparameters from SFTConfig
)

print("✅ Trainer initialized\n")

# 💡 WHAT HAPPENS DURING INITIALIZATION?
# ---------------------------------------
# 1. Verify dataset format (check "text" field exists)
# 2. Setup data collator (handles batching and padding)
# 3. Initialize optimizer (AdamW 8-bit)
# 4. Setup learning rate scheduler (cosine with warmup)
# 5. Prepare model for distributed training (if multi-GPU)
# 6. Create output directories for checkpoints and logs

#============================================================================
# 🎯 SECTION 9: TRAINING (The Main Event!)
#============================================================================
# Actually running the fine-tuning process

print("="*80)
print("Starting Training (A100 - Optimized for Accuracy)")
print("="*80)
print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(eval_dataset)}")
print(f"Epochs: {config.NUM_EPOCHS}")

# 💡 CALCULATE TRAINING STEPS
# ---------------------------
# Steps per epoch = training_samples / effective_batch_size
# Total steps = steps_per_epoch × num_epochs
steps_per_epoch = len(train_dataset) // (config.BATCH_SIZE * config.GRADIENT_ACCUMULATION_STEPS)
total_steps = steps_per_epoch * config.NUM_EPOCHS

print(f"Steps per epoch: ~{steps_per_epoch}")
print(f"Total steps: ~{total_steps}")
print("="*80 + "\n")

# 💡 EXAMPLE CALCULATION:
# -----------------------
# If you have 320 training examples:
# - Effective batch: 32
# - Steps per epoch: 320 / 32 = 10 steps
# - Total steps (7 epochs): 10 × 7 = 70 steps
#
# Timeline:
# - Logging: Every 5 steps (14 logs)
# - Evaluation: Every 25 steps (3 evaluations)
# - Checkpointing: Every 50 steps (2 checkpoints)

# Check initial memory usage
if torch.cuda.is_available():
    start_memory_gb = torch.cuda.max_memory_reserved() / 1024**3
    print(f"Initial VRAM: {start_memory_gb:.2f} GB\n")

# ═══════════════════════════════════════════════════════════════════════
# 🔥 THE ACTUAL TRAINING HAPPENS HERE
# ═══════════════════════════════════════════════════════════════════════

# 💡 WHAT HAPPENS DURING trainer.train()?
# ----------------------------------------
# For each epoch (7 total):
#   For each batch (steps_per_epoch):
#     1. Load batch of examples
#     2. Tokenize text → input IDs
#     3. Forward pass: model(input_ids) → predictions
#     4. Calculate loss: how different are predictions from targets?
#     5. Backward pass: calculate gradients (∂loss/∂parameters)
#     6. (Every 4 steps) Update LoRA parameters with accumulated gradients
#     7. (Every 5 steps) Log metrics (loss, LR, speed)
#     8. (Every 25 steps) Evaluate on validation set
#     9. (Every 50 steps) Save checkpoint if best so far
#
# Expected output during training:
# - Step 1: loss ~2.0 (random initialization)
# - Step 10: loss ~1.2 (starting to learn)
# - Step 25: loss ~0.8 (first evaluation)
# - Step 50: loss ~0.5 (first checkpoint)
# - Step 70: loss ~0.3 (converged)

trainer_stats = trainer.train()

# 💡 TRAINING COMPLETE!
# ---------------------
# trainer_stats contains:
# - training_loss: Average loss across all steps
# - train_runtime: Total training time in seconds
# - train_samples_per_second: Throughput metric
# - epochs_trained: Confirmation of 7 epochs

print("\n" + "="*80)
print("✅ Training Complete!")
print("="*80)

In [ ]:
#============================================================================
# 💾 SECTION 10: SAVING THE MODEL
#============================================================================
# Saving only the LoRA adapters (not the full model)

print("="*80)
print("Saving LoRA Adapters")
print("="*80)

# 💡 WHAT ARE WE SAVING?
# ----------------------
# We're NOT saving the entire 1.1B parameter model!
# We're ONLY saving the LoRA adapters (~40M parameters)
#
# Why?
# - Base model hasn't changed (still frozen)
# - Base model is already on HuggingFace Hub
# - Only adapters are new (what we trained)
#
# File size comparison:
# - Full model: ~2.2 GB (1.1B params in FP16)
# - LoRA adapters: ~40 MB (40M params in FP16)
# - Savings: 98% smaller! ⭐

output_path = Path(config.OUTPUT_DIR) / "lora_adapters"
output_path.mkdir(parents=True, exist_ok=True)

# Save LoRA adapter weights
model.save_pretrained(str(output_path))
# This saves: adapter_config.json, adapter_model.safetensors

# Save tokenizer (needed for inference)
tokenizer.save_pretrained(str(output_path))
# This saves: tokenizer.json, tokenizer_config.json, special_tokens_map.json

print(f"✅ LoRA adapters saved to: {output_path}")
print(f"✅ Adapter size: ~20-50 MB (vs 2.2 GB for full model)")
print(f"✅ Tokenizer saved")

In [ ]:
# 💡 SWITCHING TO INFERENCE MODE
# -------------------------------
# Training mode: Enables dropout, gradient calculation
# Inference mode: Disables dropout, no gradients (faster, deterministic)
FastLanguageModel.for_inference(model)

# ═══════════════════════════════════════════════════════════════════════
# 🔧 INFERENCE HELPER FUNCTIONS
# ═══════════════════════════════════════════════════════════════════════

def build_prompt(user_msg: str) -> str:
    """
    Build ChatML prompt for inference.

    💡 IMPORTANT: Must match training format exactly!
    ------------------------------------------------
    Training format: <|user|>\n{question}</s>\n<|assistant|>\n{answer}</s>\n
    Inference format: <|user|>\n{question}</s>\n<|assistant|>\n
                      ↑ Same prefix, model completes the rest ↑

    Why this matters:
    - Model was trained to complete text after <|assistant|> token
    - Wrong format = model won't generate properly
    - Must include </s> after user message (model expects it)
    """
    return f"<|user|>\n{user_msg}</s>\n<|assistant|>\n"

def extract_answer(full_text: str) -> str:
    """
    Extract assistant's response from generated text.

    Generated text includes prompt + completion:
    "<|user|>\nQuestion</s>\n<|assistant|>\nAnswer</s>"

    We want just the answer part after <|assistant|>.
    """
    if "<|assistant|>" not in full_text:
        return full_text  # Fallback if format is wrong

    # Split at <|assistant|> and take everything after it
    ans = full_text.split("<|assistant|>")[-1]

    # Remove </s> token and any trailing whitespace
    return ans.split(tokenizer.eos_token)[0].strip()

def generate_response(question: str, max_new_tokens: int = 120):
    """
    Generate response to a question using the fine-tuned model.

    Args:
        question: User's question (e.g., "Who won IPL 2023?")
        max_new_tokens: Maximum tokens to generate (120 = ~90 words)

    Returns:
        Generated answer text
    """
    # Build ChatML formatted prompt
    prompt = build_prompt(question)

    # Tokenize: text → token IDs
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    # 💡 GENERATION PARAMETERS:
    # -------------------------
    with torch.no_grad():  # Disable gradient calculation (faster)
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,  # Generate up to 120 new tokens

            temperature=0.1,
            # 💡 TEMPERATURE: Controls randomness
            # - 0.0: Deterministic (always pick highest probability word)
            # - 0.1: Very focused (good for factual Q&A) ⭐ WE USE THIS
            # - 0.7: Balanced (default for chat)
            # - 1.0: Creative (for story generation)
            #
            # For IPL facts, we want LOW temperature (factual answers)

            top_p=0.9,
            # 💡 TOP-P (Nucleus Sampling): Alternative to temperature
            # - Only sample from top 90% probability mass
            # - Filters out very unlikely words
            # - 0.9 = standard value

            do_sample=True,
            # Enable sampling (use temperature/top_p)
            # False = greedy decoding (always pick max probability)

            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id,
            # Stop generation when </s> token is generated
        )

    # Decode: token IDs → text
    full_text = tokenizer.decode(outputs[0], skip_special_tokens=False)

    # Extract just the answer part
    return extract_answer(full_text)

In [ ]:
# ═══════════════════════════════════════════════════════════════════════
# 🎯 TEST QUESTIONS
# ═══════════════════════════════════════════════════════════════════════

test_questions = [
    "In IPL 2023, who won when Gujarat Titans played against Chennai Super Kings?",
    "Where was the IPL 2023 match between Rajasthan Royals and Sunrisers Hyderabad played?",
    "Which stadium hosted the Punjab Kings vs Rajasthan Royals match in IPL 2023?",
    "Who was the Man of the Match when Mumbai Indians played Chennai Super Kings in IPL 2023?",
]

# 💡 WHAT TO EXPECT:
# ------------------
# Before fine-tuning: Model would give generic/wrong answers
#   Q: "Who won Gujarat Titans vs CSK in IPL 2023?"
#   A: "I don't have information about IPL 2023..." (base model)
#
# After fine-tuning: Model should give specific correct answers
#   Q: "Who won Gujarat Titans vs CSK in IPL 2023?"
#   A: "Chennai Super Kings won by 5 wickets" (fine-tuned model) ⭐

print("\n🧪 Running Test Predictions:\n")

for i, question in enumerate(test_questions, 1):
    print(f"[Test {i}/{len(test_questions)}]")
    print(f"Q: {question}")
    print("-" * 80)

    # Generate answer
    answer = generate_response(question)

    print(f"A: {answer}")
    print("=" * 80 + "\n")

In [ ]:
print("""
╔═══════════════════════════════════════════════════════════════════════╗
║                    WHAT YOU'VE LEARNED TODAY                          ║
╚═══════════════════════════════════════════════════════════════════════╝

1️⃣  FINE-TUNING vs PRE-TRAINING
   ✅ Pre-training: Teaching language from scratch (months, billions of tokens)
   ✅ Fine-tuning: Teaching specific knowledge (minutes, thousands of examples)

2️⃣  QLoRA (Quantized Low-Rank Adaptation)
   ✅ 4-bit quantization: 75% memory savings (2.2GB → 550MB base model)
   ✅ LoRA adapters: Only train 4% of parameters (1.1B → 40M trainable)
   ✅ Memory efficient: Train on single GPU vs 8 GPUs for full fine-tuning

3️⃣  HYPERPARAMETER OPTIMIZATION
   ✅ LoRA rank: Higher rank = more capacity (8 basic, 32 high, 64 overkill)
   ✅ Batch size: Effective batch 32 = stable training
   ✅ Learning rate: 2e-4 standard for LoRA, with 10% warmup
   ✅ Epochs: 7 epochs sufficient for small dataset memorization

4️⃣  DATA FORMATTING
   ✅ ChatML format: TinyLlama's conversation structure
   ✅ Consistency: Training and inference formats must match!
   ✅ Validation split: 10% held out to monitor overfitting

5️⃣  EVALUATION & DEPLOYMENT
   ✅ Test on specific questions to verify learning
   ✅ Save only adapters (~40 MB vs 2.2 GB full model)
   ✅ Easy deployment: Load base model + adapters

╔═══════════════════════════════════════════════════════════════════════╗
║                       KEY NUMBERS TO REMEMBER                         ║
╚═══════════════════════════════════════════════════════════════════════╝

📊 Model Size:
   - Full model: 1.1B parameters (2.2 GB in FP16)
   - Quantized: 1.1B parameters (550 MB in 4-bit)
   - LoRA adapters: 40M parameters (40 MB in FP16)
   - Trainable: 3-4% of total parameters

⏱️  Training Time:
   - A100 GPU: ~10-15 minutes
   - T4 GPU: ~25-35 minutes (with adjusted config)
   - Without Unsloth: ~2-3 hours
   - Speedup: 8-12x faster!

💾 Memory Usage:
   - A100 (this config): ~12-15 GB VRAM
   - T4 (adjusted): ~10-12 GB VRAM
   - Full fine-tuning: ~35-40 GB VRAM
   - Savings: 60-70% less memory

🎯 Quality Metrics:
   - Final training loss: ~0.3-0.5 (excellent)
   - Validation loss: Should track training (no overfitting)
   - Inference: Specific, factual answers to IPL 2023 questions

╔═══════════════════════════════════════════════════════════════════════╗
║                    WORKSHOP DISCUSSION POINTS                         ║
╚═══════════════════════════════════════════════════════════════════════╝

❓ QUESTION 1: Why use LoRA instead of full fine-tuning?
   💡 Answer:
   - Memory: 60-70% less VRAM (enables training on consumer GPUs)
   - Speed: 2-5x faster training
   - Storage: 98% smaller adapter files (40 MB vs 2.2 GB)
   - Flexibility: Can train multiple adapters for different tasks

❓ QUESTION 2: Why is rank 32 better than rank 8 for this task?
   💡 Answer:
   - Rank = capacity to memorize facts
   - IPL 2023 has many specific facts (teams, venues, players, scores)
   - Rank 8: Can memorize ~10-20 core facts (basic)
   - Rank 32: Can memorize ~100+ facts (comprehensive)
   - Trade-off: Rank 32 is 4x slower to train but much more accurate

❓ QUESTION 3: What if I want to fine-tune on T4 GPU instead of A100?
   💡 Answer:
   - Reduce MAX_SEQ_LENGTH: 2048 → 1024 (shorter sequences)
   - Reduce BATCH_SIZE: 8 → 2-4 (less memory per batch)
   - Increase GRADIENT_ACCUMULATION_STEPS: 4 → 8-16 (keep effective batch 32)
   - Consider reducing rank: 32 → 16 (if memory is tight)
   - Use FP16 instead of BF16 (T4 doesn't support BF16)
   - Expect ~25-35 minutes training time

❓ QUESTION 4: How do I know if my model is overfitting?
   💡 Answer:
   - Check if training loss keeps decreasing but validation loss increases
   - Compare answers on validation set vs training set
   - If model memorizes exact training examples but fails on similar questions
   - Solutions: Increase dropout, reduce epochs, add more training data

❓ QUESTION 5: Can I fine-tune on my own dataset?
   💡 Answer: YES! Just ensure:
   - Format: JSON with "instruction" and "output" fields
   - Conversion: Use to_chatml() function for TinyLlama
   - Size: 100-10,000 examples (small datasets work well)
   - Quality: Clean, accurate, consistent data
   - Validation: Hold out 10% for evaluation

╔═══════════════════════════════════════════════════════════════════════╗
║                          NEXT STEPS                                   ║
╚═══════════════════════════════════════════════════════════════════════╝

🚀 EXPERIMENT 1: Try different LoRA ranks
   - Rank 8: Fast but basic
   - Rank 16: Balanced
   - Rank 32: High accuracy (current)
   - Rank 64: Overkill (diminishing returns)
   - Compare final loss and answer quality

🚀 EXPERIMENT 2: Test on your own dataset
   - Collect 100-500 question-answer pairs
   - Format as {"instruction": "...", "output": "..."}
   - Run this same script
   - Evaluate on held-out questions

🚀 EXPERIMENT 3: Try different models
   - TinyLlama: 1.1B params (fast, good for learning)
   - Mistral-7B: 7B params (better quality, needs more VRAM)
   - Llama-3-8B: 8B params (state-of-the-art)
   - Note: Each model has different prompt format!

🚀 EXPERIMENT 4: Hyperparameter tuning
   - Try learning rate: 1e-4, 2e-4, 5e-4
   - Try epochs: 3, 5, 7, 10
   - Try effective batch size: 16, 32, 64
   - Track validation loss to find optimal settings

╔═══════════════════════════════════════════════════════════════════════╗
║                         RESOURCES                                     ║
╚═══════════════════════════════════════════════════════════════════════╝

📚 Papers:
   - QLoRA: https://arxiv.org/abs/2305.14314
   - LoRA: https://arxiv.org/abs/2106.09685
   - LLM Fine-tuning: https://arxiv.org/abs/2403.14608

🔧 Tools:
   - Unsloth: https://github.com/unslothai/unsloth
   - HuggingFace PEFT: https://github.com/huggingface/peft
   - TRL (SFTTrainer): https://github.com/huggingface/trl

💬 Community:
   - Unsloth Discord: discord.gg/unsloth
   - HuggingFace Forum: discuss.huggingface.co
   - r/LocalLLaMA: reddit.com/r/LocalLLaMA

╔═══════════════════════════════════════════════════════════════════════╗
║                    THANK YOU FOR ATTENDING!                           ║
╚═══════════════════════════════════════════════════════════════════════╝

Questions? Let's discuss! 🎤
""")

print("="*80)
print("Workshop script execution complete!")
print("="*80)